In [73]:
import anthropic
import os
from dotenv import load_dotenv
load_dotenv()
client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)
# print(client.api_key)
message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    temperature=0.0,
    system="translate the input into English and respond in Hebrew",
    messages=[
        {"role": "user", "content": "מה קורה?"}
    ]
)

print(message.content)

[TextBlock(text='שלום! אני שמח לעזור לך. מה אפשר לעזור לך?', type='text')]


In [74]:
def get_response(input):
    message = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.0,
        system="answer in Hebrew to the user question",
        messages=[
            {"role": "user", "content": input}
        ]
    )
    return message.content

In [75]:
print(get_response("ספר לי על המלחמה של ישראל וחמאס בשנת 2023"))

[TextBlock(text='להלן סיכום של המלחמה בין ישראל וחמאס בשנת 2023:\n\n• הסלמה החלה בעקבות עימותים באזור ירושלים והר הבית, שהובילו לעימותים צבאיים בין ישראל לחמאס ברצועת עזה.\n\n• חמאס ירה מאות רקטות לעבר ערים בישראל, בעוד ישראל תקפה מטרות צבאיות ברצועה בתגובה.\n\n• הלחימה נמשכה כשבועיים, תוך הרס נרחב בעזה והרוגים משני הצדדים. ישראל הפעילה את מערכת ההגנה האווירית "כיפת ברזל" להתמודדות עם הרקטות.\n\n• בסופו של דבר, הושג הסכם הפסקת אש בתיווך בינלאומי. הצדדים הסכימו על הפסקת אש, תמורת הקלות על המצור הישראלי על עזה ושיקום התשתיות שנהרסו.\n\n• המלחמה הביאה לנזקים כבדים בעזה ואבדות אנושיות משני הצדדים, אך לא הובילה לפריצת דרך משמעותית בסכסוך הישראלי-פלסטיני.', type='text')]


In [76]:
def translate_into_hebrew(input_text):
    message = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=4000,
        temperature=0.0,
        system="translate the input into Hebrew return the translated text",
        messages=[
            {"role": "user", "content": input_text}
        ]
    )
    return message.content

In [77]:
import PyPDF2
def extract_pages_as_text_in_list(file_path):
    reader=PyPDF2.PdfReader(file_path)
    pages=[]
    for page in reader.pages:
        pages.append(page.extract_text())
    return pages
def extract_all_text(file_path):
    reader=PyPDF2.PdfReader(file_path)
    text=""
    for page in reader.pages:
        text+=page.extract_text()
    return text


In [78]:
pages=extract_pages_as_text_in_list("/Users/matansharon/python/Data_science/some tests/ivrit_ai.pdf")

In [79]:
translated_text=translate_into_hebrew(pages[2])

In [80]:
import pandas as pd
import anthropic
import os
def analyze_excel(file_path):
    
    df=pd.read_excel(file_path)
    df_as_string=df.to_string()
    client=anthropic.Anthropic(
        api_key=os.environ.get("ANTHROPIC_API_KEY")
    )
    
    message = client.messages.create(
        # model='claude-3-haiku-20240307',
        model='claude-3-opus-20240229',
        max_tokens=4096,
        temperature=0.2,
        system="""t
        this is an excel file that contains data about the Elcam's medical company.
        each row is an idea came up in a meeting by Elcam's board of directors and management team.
        please analyze the data and provide insights about the data.
        after that create a calculation with a recommendation score for each idea.
        and finally, print the data in a table format with the recommendation score order by the highest recommendation score .
        
        """,
        messages=[
            {"role": "user", "content": df_as_string}
        ]
    )
    return message.content[0].text

def summarize_text(file_path):
    text=extract_all_text(file_path)
    client=anthropic.Anthropic(
        api_key=os.environ.get("ANTHROPIC_API_KEY")
    )
    message = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.0,
        system="summarize the text",
        messages=[
            {"role": "user", "content": text}
        ]
    )
    return message.content
def visulize_data(input_text):
    client=anthropic.Anthropic(
        api_key=os.environ.get("ANTHROPIC_API_KEY")
    )
    message = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1000,
        temperature=0.0,
        system="""
        this is a summerztion of the data of an excel file that contains data about the Elcam's medical company.
        at the end there is a table with the recommendation score order by the highest recommendation score.
        write a code that will create a visualization of the data.
        """,
        messages=[
            {"role": "user", "content": input_text}
        ]
    )
    return message.content

In [81]:

s=analyze_excel('ideas.xlsx')


In [82]:
print(s)

Here are the key insights and analysis of the data:

1. The ideas span various categories related to Elcam's supply chain, including end-to-end supply chain, reducing bureaucracy, suppliers, customers, forecasting, orders, production planning, inventory, shipments, and product development.

2. Many ideas focus on automation, digitization and using AI/algorithms to streamline processes, reduce manual work, improve efficiency and provide better service to customers. Examples include automated order entry, production scheduling, inventory management, and shipment tracking.

3. Several ideas aim to provide more transparency and information sharing with suppliers and customers to improve planning, reduce lead times and enhance flexibility. This includes supplier portals, customer service chatbots, managing customer inventory, etc.

4. Improving forecasting accuracy by leveraging internal and external data is another common theme. Ideas propose using AI to analyze multiple data sources for b

In [83]:
print(s)

Here are the key insights and analysis of the data:

1. The ideas span various categories related to Elcam's supply chain, including end-to-end supply chain, reducing bureaucracy, suppliers, customers, forecasting, orders, production planning, inventory, shipments, and product development.

2. Many ideas focus on automation, digitization and using AI/algorithms to streamline processes, reduce manual work, improve efficiency and provide better service to customers. Examples include automated order entry, production scheduling, inventory management, and shipment tracking.

3. Several ideas aim to provide more transparency and information sharing with suppliers and customers to improve planning, reduce lead times and enhance flexibility. This includes supplier portals, customer service chatbots, managing customer inventory, etc.

4. Improving forecasting accuracy by leveraging internal and external data is another common theme. Ideas propose using AI to analyze multiple data sources for b

In [84]:

vs=visulize_data(s)


In [85]:
print(vs[0].text)

To visualize the data effectively, I recommend creating an interactive dashboard using a tool like Tableau or Power BI. The dashboard can include the following visualizations:

1. A bar chart showing the number of ideas in each category. This will provide an overview of which areas have the most suggestions for improvement.

2. A word cloud of the most frequently occurring keywords in the idea names. This will highlight the main themes and focus areas across all ideas.

3. A scatter plot with the idea number on the x-axis and the recommendation score on the y-axis. This will show the distribution of scores and identify the top-rated ideas. You can add category labels or colors to see if certain categories tend to have higher scores.

4. A table with the top 10 or 20 ideas by recommendation score, including the category, idea name, and score columns. This provides a quick view of the best ideas.

5. A timeline or roadmap visualization showing the proposed implementation timeline for the

In [86]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='In the realm where code runs wild and free,\nThere lies a concept both subtle and key,\nRecursion, a loop that bends back on itself,\nA dance of function and call, like books on a shelf.\n\nAt its heart, a function that calls its own name,\nA loop within a loop, a recursive game,\nLike a mirror reflecting an image anew,\nIt loops and it loops, a mesmerizing view.\n\nWith each call, a new instance is born,\nA task partially completed, waiting to adorn,\nThe next step in the dance, the code’s graceful glide,\nRecursion whispers softly, come along for the ride.\n\nUntil a base case halts the endless loop,\nThe final curtain falls, the program’s last sloop,\nRecursion fades away, leaving memories to keep,\nA complex concept unraveled, in the code’s deep sleep.\n\nSo embrace recursion, let it weave its spell,\nIn the world of code, where mysteries dwell,\nA loop within a loop, a dance of endless grace,\nRecursion whispers softly, in the programmer’s embrace.',